In [1]:
import requests
import pandas as pd
import numpy as np
import os

In [2]:
codes=['SiZ0','SiH1','SiM1','SiU1','SiZ1']

In [ ]:
#get boardid
url_boardid='https://iss.moex.com/iss/engines/futures/markets/forts/boards.csv'
boardid=pd.read_csv(url_boardid, sep=';',nrows=1,skiprows=1, encoding='windows-1251').boardid.iloc[0]

url_forts_sec='https://iss.moex.com/iss/engines/futures/markets/forts/boards/'+boardid+'/securities.json'
dict_forts_securities=requests.get(url_forts_sec).json()
print(dict_forts_securities['securities']['data'][0])
print(dict_forts_securities['securities']['columns'])
print(type(dict_forts_securities['securities']['data'][0]))

In [ ]:
# create dataframe from json
df=pd.DataFrame(dict_forts_securities['securities']['data'],columns=dict_forts_securities['securities']['columns'])
print(df.info())
df=df.reset_index().drop(['index'],axis=1)

In [ ]:
df[df['SECID'].str.contains(r'^SR(?!$)')]
df[df['SECID'].str.contains('SR',regex=False)]

In [2]:
def f_get_trading_dates(code):
    #get boardid
    url_boardid='https://iss.moex.com/iss/engines/futures/markets/forts/boards.csv'
    boardid=pd.read_csv(url_boardid, sep=';',nrows=1,skiprows=1, encoding='windows-1251').boardid.iloc[0]
    #print('| code = '+code+' | boardid = '+boardid+' |')

    # запрос на торговый диапазон дат по инструменту
    url_candleborders='https://iss.moex.com/iss/engines/futures/markets/forts/boards/'+boardid+'/securities/'+code+'/candleborders.csv'
    print('get url: ' + url_candleborders)
    # запрашиваем данные по торговому диапазону дат инструмента и записываем в датафрейм
    df_candleborders=pd.read_csv(url_candleborders,sep=';',nrows=1,skiprows=1)
    #print(df_candleborders)

    #extract begin date from dataframe
    date_first=df_candleborders.begin.str[:10][0]
    date_last=df_candleborders.end.str[:10][0]
    interval=df_candleborders.interval[0]

    dictionary = {code: {'date_first': date_first, 'date_last': date_last, 'interval': interval}}

    print('code='+code+'\ninterval='+str(interval)+' min\ndate_begin='+date_first+'\ndate_end='+date_last+'\n------------------')
    
    return(dictionary)

In [8]:
dictionary=f_get_trading_dates(code='SiH1')

| code = SiH1 | boardid = RFUD |
get url: https://iss.moex.com/iss/engines/futures/markets/forts/boards/RFUD/securities/SiH1/candleborders.csv
| interval = 1min | date_start = 2019-03-18 | date_end = 2021-03-18| 
------------------


In [7]:
print(os.getcwd())

d:\vscode\projects\IIS_Moex


In [11]:
#wdir=('d:\\Anaconda3\\My_scripts\\IIS_Moex\\historyIIS\\')
if os.getcwd()!='d:\\Anaconda3\\My_scripts\\IIS_Moex\\historyIIS\\': os.chdir('d:\\Anaconda3\\My_scripts\\IIS_Moex\\historyIIS\\')
print(os.getcwd())

d:\Anaconda3\My_scripts\IIS_Moex\historyIIS


In [40]:
def f_get_data_from_IIS_MOEX(code,date_first,date_last):

    date_list=[d.strftime('%Y-%m-%d') for d in pd.date_range(date_first,date_last)]
    #создаем папку для сохранения данных
    folder = code+'_'+ date_first +'_'+ date_last
    #folder=code+'_'+dictionary['SiH1']['date_end_prev']+'_'+dictionary['SiH1']['date_end']
    if not os.path.exists(folder): 
            os.makedirs(folder)
            print('| Folder to save data: ' + os.getcwd()+'\\' + folder + ' |')

    print('| Start extract data from server |\n| '+code+' | from '+ date_first +' | till '+ date_last +' |\n'+45*'-')

    # цикл диапазона торгуемых дат по инструменту
    num_of_days=0
    #----------------------------------------------------------------------------------------------------------------------------
    for date in date_list:
        print('| Get data from server: | '+code+' | '+date+' |')
        num_of_days=num_of_days + 1
        #----------------------------------------------------------------------------------------------------------------------------
        #максимальный объем данных запроса с сервера - 500 строк. Чтобы извлечь все данные необходимо увеличевать значение size в цикле
        for size in np.arange(0,501,500):
            
            #формируем url запрос и добавляем переменную start, т.е. с какой строки выполнить запрос занных с сервера 
            url_candles = 'https://iss.moex.com/iss/engines/futures/markets/forts/boards/FOB/'\
                'securities/'+code+'/candles.csv?interval=1&from='+date+'&till='+ date +'&start='+str(size)
            #делаем url запрос и сохраняем полученные данные в датафрейм
            df_url_candles = pd.read_csv(url_candles,sep=';',header=1)

            #если за указанную дату нет данных, то переходим к следующей дате
            if len(df_url_candles)==0 and size==0:
                #print('! no data for date | '+ code +' | '+ date +' | go to next date |')
                num_of_days=num_of_days - 1
                break
            
            try:
                len(df_hist.index)
            except:
                df_hist=df_url_candles.copy()
            
            if len(df_hist.index)==0:
                df_hist=df_url_candles.copy()
            elif size!=0:
                df_hist=df_hist.append(df_url_candles)
            
            pd.DataFrame.to_csv(df_hist, os.getcwd() +'\\'+ folder + '\\'+code+ '_'+date+'.txt',index=False,sep=';')
        #print(num_of_days*'.')
        #----------------------------------------------------------------------------------------------------------------------------
        #oчищаем данные
        df_hist=df_hist.iloc[0:0]
    #----------------------------------------------------------------------------------------------------------------------------   
    print('Finished extract data'+'\nNumber of trading days: '+str(num_of_days))

In [41]:
f_get_data_from_IIS_MOEX( code='SiH1'
                         ,date_first='2020-12-16'
                         ,date_last='2021-03-18' )

| Start extract data from server |
| SiH1 | from 2020-12-16 | till 2021-03-18 |
---------------------------------------------
| Get data from server: | SiH1 | 2020-12-16 |
| Get data from server: | SiH1 | 2020-12-17 |
| Get data from server: | SiH1 | 2020-12-18 |
| Get data from server: | SiH1 | 2020-12-19 |
| Get data from server: | SiH1 | 2020-12-20 |
| Get data from server: | SiH1 | 2020-12-21 |
| Get data from server: | SiH1 | 2020-12-22 |
| Get data from server: | SiH1 | 2020-12-23 |
| Get data from server: | SiH1 | 2020-12-24 |
| Get data from server: | SiH1 | 2020-12-25 |
| Get data from server: | SiH1 | 2020-12-26 |
| Get data from server: | SiH1 | 2020-12-27 |
| Get data from server: | SiH1 | 2020-12-28 |
| Get data from server: | SiH1 | 2020-12-29 |
| Get data from server: | SiH1 | 2020-12-30 |
| Get data from server: | SiH1 | 2020-12-31 |
| Get data from server: | SiH1 | 2021-01-01 |
| Get data from server: | SiH1 | 2021-01-02 |
| Get data from server: | SiH1 | 2021-01-03 |


In [4]:
#get max and min date for contracts
code='SiH1'
url_candleborders='https://iss.moex.com/iss/engines/futures/markets/forts/boards/RFUD/securities/'+code+'/candleborders.csv'
print(url_candleborders)
df_candleborders=pd.read_csv(url_candleborders,sep=';',nrows=1,skiprows=1)
print(df_candleborders)

https://iss.moex.com/iss/engines/futures/markets/forts/boards/RFUD/securities/SiH1/candleborders.csv
                 begin                  end  interval
0  2019-03-18 20:48:00  2021-03-18 13:59:00         1


In [10]:
'''
if 'dict' in locals():
  print('exists')
date_end_prev=None
if date_end_prev is None:
  print('date_end_prev var is None')

if 'date_end_prev' not in locals():
  print('error')
'''

error


In [5]:

#df = pd.DataFrame(columns=['code', 'date_begin', 'date_end','date_end_prev'])
#dictionary={}
for code in codes:
    print(code)
    # запрос на торговый диапазон дат по инструменту
    url_candleborders='https://iss.moex.com/iss/engines/futures/markets/forts/boards/RFUD/securities/'+code+'/candleborders.csv'
    print('get url: ' + url_candleborders)
    # запрашиваем данные по торговому диапазону дат инструмента и записываем в датафрейм
    df_candleborders=pd.read_csv(url_candleborders,sep=';',nrows=1,skiprows=1)
    #print(df_candleborders)

    if 'date_end_prev' not in locals():
        date_end_prev=df_candleborders.end.str[:10][0]
    else:
        date_end_prev=date_end

    #extract begin date from dataframe
    date_begin=df_candleborders.begin.str[:10][0]
    date_end=df_candleborders.end.str[:10][0]
    interval=df_candleborders.interval[0]

    if not dictionary:
        dictionary = {code: {'date_begin': date_begin, 'date_end_prev': date_end_prev, 'date_end': date_end}}
    else:
        dictionary.update({code: {'date_begin': date_begin, 'date_end_prev': date_end_prev, 'date_end': date_end}})
    '''
    if 'dict' in locals():
        dict.update({code: {'date_begin': date_begin, 'date_end_prev': date_end_prev, 'date_end': date_end}})
    else: dict = {code: {'date_begin': date_begin, 'date_end_prev': date_end_prev, 'date_end': date_end}}
    
    df = df.append({'code' : code, 'date_begin' : date_begin, 'date_end_prev' : date_end_prev, 'date_end' : date_end}, 
                ignore_index = True)
    '''
    print('code='+code+'\ninterval='+str(interval)+' min\ndate_begin='+date_begin+'\ndate_end='+date_end+'\n------------------')

del date_begin,date_end


SiZ0
https://iss.moex.com/iss/engines/futures/markets/forts/boards/RFUD/securities/SiZ0/candleborders.csv
code=SiZ0
interval=1
date_begin=2018-12-19
date_end=2020-12-17
------------------
SiH1
https://iss.moex.com/iss/engines/futures/markets/forts/boards/RFUD/securities/SiH1/candleborders.csv
code=SiH1
interval=1
date_begin=2019-03-18
date_end=2021-03-18
------------------
SiM1
https://iss.moex.com/iss/engines/futures/markets/forts/boards/RFUD/securities/SiM1/candleborders.csv
code=SiM1
interval=1
date_begin=2019-06-20
date_end=2021-06-17
------------------
SiU1
https://iss.moex.com/iss/engines/futures/markets/forts/boards/RFUD/securities/SiU1/candleborders.csv
code=SiU1
interval=1
date_begin=2019-09-16
date_end=2021-09-16
------------------
SiZ1
https://iss.moex.com/iss/engines/futures/markets/forts/boards/RFUD/securities/SiZ1/candleborders.csv
code=SiZ1
interval=1
date_begin=2019-12-13
date_end=2021-12-16
------------------
{'SiZ0': {'date_begin': '2018-12-19', 'date_end_prev': '2020

In [6]:
code='SiH1'
dictionary[code]
#dictionary['SiH1']

{'date_begin': '2019-03-18',
 'date_end_prev': '2020-12-17',
 'date_end': '2021-03-18',
 'interval': 1}

In [8]:
date_list=[d.strftime('%Y-%m-%d') for d in pd.date_range(dictionary[code]['date_end_prev'],dictionary[code]['date_end'])]
#date_list=[d.strftime('%Y-%m-%d') for d in pd.date_range(dictionary['SiH1']['date_end_prev'],dictionary['SiH1']['date_end'])]
print(date_list)


['2020-12-17', '2020-12-18', '2020-12-19', '2020-12-20', '2020-12-21', '2020-12-22', '2020-12-23', '2020-12-24', '2020-12-25', '2020-12-26', '2020-12-27', '2020-12-28', '2020-12-29', '2020-12-30', '2020-12-31', '2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08', '2021-01-09', '2021-01-10', '2021-01-11', '2021-01-12', '2021-01-13', '2021-01-14', '2021-01-15', '2021-01-16', '2021-01-17', '2021-01-18', '2021-01-19', '2021-01-20', '2021-01-21', '2021-01-22', '2021-01-23', '2021-01-24', '2021-01-25', '2021-01-26', '2021-01-27', '2021-01-28', '2021-01-29', '2021-01-30', '2021-01-31', '2021-02-01', '2021-02-02', '2021-02-03', '2021-02-04', '2021-02-05', '2021-02-06', '2021-02-07', '2021-02-08', '2021-02-09', '2021-02-10', '2021-02-11', '2021-02-12', '2021-02-13', '2021-02-14', '2021-02-15', '2021-02-16', '2021-02-17', '2021-02-18', '2021-02-19', '2021-02-20', '2021-02-21', '2021-02-22', '2021-02-23', '2021-02-24', '2021-02-25', '2021

In [9]:
#создаем папку для сохранения данных
folder = code+'_'+dictionary[code]['date_end_prev']+'_'+dictionary[code]['date_end']
#folder=code+'_'+dictionary['SiH1']['date_end_prev']+'_'+dictionary['SiH1']['date_end']
if not os.path.exists(folder): 
        os.makedirs(folder)
        print('folder to save data : ' + folder )

print(' | '+code+' | '+dictionary[code]['date_end_prev']+' | '+dictionary[code]['date_end']+'\n------------------')

# цикл диапазона торгуемых дат по инструменту
for date in date_list:
    print('get data from server: | '+code+' | '+date+' |')
    #url для запроса get данных
    url_candles = 'https://iss.moex.com/iss/engines/futures/markets/forts/boards/FOB/'\
    'securities/'+code+'/candles.csv?interval=1&from='+date+'&till='+ date +'&start='

    #максимальный объем данных - 500 строк. Чтобы извлечь все данные необходимо увеличевать значение size в цикле
    for size in np.arange(0,501,500):
        
        #добавляем к url запросу переменную start, т.е. с какой строки выполнить запрос занных с сервера 
        url_candles=url_candles+str(size)
        #делаем url запрос и сохраняем полученные данные в датафрейм
        df_url_candles = pd.read_csv(url_candles,sep=';',header=1)

        #если за указанную дату нет данных, то переходим к следующей дате
        if len(df_url_candles)==0:
            print('! no data for date | '+code+' | '+date+' | go to next date |')
            break
        
        try:
            len(df_hist.index)
        except:
            df_hist=df_url_candles.copy()
        
        if len(df_hist.index)==0:
            df_hist=df_url_candles.copy()
        elif size!=0:
            df_hist=df_hist.append(df_url_candles)

        pd.DataFrame.to_csv(df_hist, os.getcwd() +'\\'+ folder + '\\'+code+ '_'+date+'.txt',index=False,sep=';')
        #pd.DataFrame.to_csv(df_hist,os.getcwd() + folder +'\\'+dateFrom+'_'+code+ '.txt',index=False,sep=';')
    #print(df_url_candles)
    #print(len(df_url_candles))

 | SiH1 | 2020-12-17 | 2021-03-18
------------------
get data from server: | SiH1 | 2020-12-17 |
get data from server: | SiH1 | 2020-12-18 |
get data from server: | SiH1 | 2020-12-19 |
! no data for date 2020-12-19 | go to next date |
get data from server: | SiH1 | 2020-12-20 |
! no data for date 2020-12-20 | go to next date |
get data from server: | SiH1 | 2020-12-21 |
get data from server: | SiH1 | 2020-12-22 |
get data from server: | SiH1 | 2020-12-23 |
get data from server: | SiH1 | 2020-12-24 |
get data from server: | SiH1 | 2020-12-25 |
get data from server: | SiH1 | 2020-12-26 |
! no data for date 2020-12-26 | go to next date |
get data from server: | SiH1 | 2020-12-27 |
! no data for date 2020-12-27 | go to next date |
get data from server: | SiH1 | 2020-12-28 |
get data from server: | SiH1 | 2020-12-29 |
get data from server: | SiH1 | 2020-12-30 |
get data from server: | SiH1 | 2020-12-31 |
! no data for date 2020-12-31 | go to next date |
get data from server: | SiH1 | 2021-0

In [102]:
code='SiH1'
dateFrom='2020-12-19'
dateTill='2020-12-19'
url_candles = 'https://iss.moex.com/iss/engines/futures/markets/forts/boards/FOB/'\
    'securities/'+code+'/candles.csv?interval=1&from='+dateFrom+'&till='+ dateTill +'&start=0'
df_url_candles=pd.read_csv(url_candles,sep=';',header=1)
print(df_url_candles)
print(len(df_url_candles))


Empty DataFrame
Columns: [open, close, high, low, value, volume, begin, end]
Index: []
0
